#### Analysis of the phishing data set
Prediction is too good, probably includes training set in training data somehow...

Just testing out the markup, for example math expressions: <br>
$d = \sqrt{x_1^2 +x_2^2}$ <br>
We start out with loading the data, setting some config variables and splitting the data

In [1]:
from random import randrange
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Config
evaluation_method = "holdout" #holdout or cross
holdout_split = 0.8
cross_validations = 10

# Load data
df_raw = pd.read_csv("data/phpV5QYya.csv")

# Preprocess
# Data is already preprocessed - just shuffle to make sure
df_shuffeled = df_raw.sample(frac=1,random_state=193520285)

#Split data
subsets = []
if(evaluation_method == "holdout"):
    num_learning = round(df_shuffeled.shape[0] * holdout_split)
    subsets.append(df_shuffeled[0:num_learning-1])
    subsets.append(df_shuffeled[num_learning:])
    
if(evaluation_method == "cross"):
    num_per_subset = (int)(df_shuffeled.shape[0]/cross_validations)
    residual = df_shuffeled.shape[0]%cross_validations

    offset = 0
    for i in range(cross_validations):
        begin = num_per_subset*i+offset
        if(offset<residual):
            offset = offset+1
        end = num_per_subset*(i+1)+offset

        subsets.append(df_shuffeled[begin:end])

First we set up the evaluation method:

In [ ]:
for row in subsets[0].iterrows():
    x= sum(row[1][:-1])
    #print(x)

In [39]:
def evaluate(model):
    print(f"Evaluating with method: {evaluation_method}")
    
    if(evaluation_method=="holdout"):
        model.train(subsets[0])
        
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        for row in subsets[1].iterrows():
            instance = row[1]
            pred = model.predict(instance)
            
            if(instance['Result']==1):
                if(pred==1):
                    tp = tp+1
                if(pred==-1):
                    fn = fn+1
            if(instance['Result']==-1):
                if(pred==1):
                    fp = fp+1
                if(pred==-1):
                    tn = tn+1
                
            print(f"sens: {(tp/(tp+fn))}, acc: { (tp+tn)/(tp+tn+fp+fn)}")
        return (tp/(tp+fn)), (tp+tn)/(tp+tn+fp+fn)
                
        

In [31]:
#Assumes last element in each row is the target
#Assumes classes = {1 (=phising), -1(=safe)}

class kNN:
    def __init__(self, k):
        self.k = k
        
    def train(self, data):
        #Just save the entire dataset
        self.data = data
        
    def predict(self, instance):
        #neighbours is list of tuple( target_vote, distance )
        #Sorted smallest distance -> highest
        neighbours = []
        
        #Compare instance to every element in data set
        for row in self.data.iterrows():
            element = row[1]
            
            #Calculate distance from instance to element
            dist = sum((instance[:-1] - element[:-1]).abs())
            
            #If first element, just add 
            if(len(neighbours)==0):
                neighbours.append((element['Result'], dist))
            else:
                
                for i, n in enumerate(neighbours):
                    #If the calculated distance is shorter than compared neighbour, insert _before_ this neighbour
                    if(dist<n[1]):
                        neighbours.insert(i,(element['Result'],dist))
                        
                    #If we have to many neighbours, remove last one in list (with highest distance)
                    if(len(neighbours)>self.k):
                        del neighbours[-1]
             
        #Perform count of each class (1 or -1)
        phis_count = 0
        safe_count = 0
        for n in neighbours:
            if(n[0]==1):
                phis_count = phis_count + 1
            if(n[0]==-1):
                safe_count = safe_count + 1
        
        #Return 
        if (phis_count >= safe_count):
            return 1
        else: 
            return -1
    
    

In [ ]:
print("hello")

kNN3 = kNN(3)

result = evaluate(kNN3)

print(result)
print(f"Accuarcy: {result[1]}, sensitivity: {result[0]}")